# Miniproject 2

In [ ]:

!pip install backports.cached_property

# Setup matplotlib animation
import matplotlib
matplotlib.rc('animation', html='jshtml')


## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from typing import Optional


def implementation_for(cls, method_name: Optional[str] = None):
    """Helper to provide a concrete implementation for a class method.

    This utility allows one to define methods outside of a class defintion.
    Thus allowing one to provide the implementations of a class "incrementally". 
    This utility is useful for gradually submitting class method implementations 
    to catsoop. 

    It is intended to be used as a decorator.

    Args:
        cls: a Python class on which we are implementing the method.
        method_name: an optional method name to indicate which method we are 
            implementing. If not provided, infer the method name by using the 
            decorated function's name.

    Example:
        >>> class A: 
        ...   def foo(self):
        ...     raise NotImplementedError()

        >>> @implementation_for(A)
        ... def foo(self):
        ...   print("implemented foo!")

        >>> A().foo()
        implemented foo!
    """

    def decorator(meth: Callable) -> Callable:
        mname = method_name or meth.__name__
        setattr(cls, mname, meth)
        return meth

    return decorator



from typing import (Callable, Iterable, List, Sequence, Optional, Dict)

from abc import abstractmethod, ABCMeta
import itertools
import math
import random
import dataclasses
from collections import namedtuple
import copy

import numpy as np

import scipy.stats
import scipy.special

# Basic data structures.

# x, y coordinate and the rotation of the robot.
Pose = namedtuple("Pose", ["x", "y", "theta"])
# unique id (str) and x, y coordinate of the landmark.
Landmark = namedtuple("Landmark", ["id", "x", "y"])
# r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Measurement = namedtuple("Measurement", ["landmark_id", "r", "b"])
# delta_p --- distance to move; delta_theta --- rotation of the robot
Command = namedtuple("Command", ["delta_p", "delta_theta"])


def grid_of_landmarks(
        x_range: Iterable[float] = range(-4, 20, 4),
        y_range: Iterable[float] = range(-4, 20, 4),
) -> Sequence[Landmark]:
    """Constructs a grid of landmarks from a catesion product of x and y coordinates."""
    return tuple(
        Landmark(f"landmark-{i}", *loc)
        for i, loc in enumerate(itertools.product(x_range, y_range)))


class Inference(metaclass=ABCMeta):
    """Interface for an inference algorithm (localization or SLAM variants)."""

    @abstractmethod
    def estimated_pose(self) -> Pose:
        """Returns the current estimation of the robot's Pose."""
        ...

    @abstractmethod
    def estimated_landmarks(self) -> Sequence[Landmark]:
        """Returns the current estimation of the landmarks."""
        ...

    @abstractmethod
    def init(self, init_state: Pose) -> None:
        """Initialize the inference with an initial pose."""
        ...

    @abstractmethod
    def update(self, measurements: Sequence[Measurement]) -> None:
        """An update step of the inference, called after every simulation step."""
        ...

    @abstractmethod
    def plot_state(self, ax) -> None:
        """Helper to visualize the internal state of the inference algorithm."""
        ...


def normalize_angles(angles: np.array) -> np.ndarray:
    """Given an array of angles in radians, element-wise normalize the angles to the range [-pi, pi].

    Args:
        angles: array of any shape.

    Returns:
        normalized angles, array of same shape as the input.
    """
    return np.arctan2(np.sin(angles), np.cos(angles))


def circular_mean(angles: np.array, axis=None) -> float:
    """Given an array of angles in radians, find the [circular mean](https://en.wikipedia.org/wiki/Circular_mean).

    Args:
        angles: array of any shape.
        axis: axis or axes along which the means are computed. The default is to compute the mean of the flattened array.

    Returns:
        mean of the angles, normalized to [-pi, pi].
    """
    return np.arctan2(np.sum(np.sin(angles), axis=axis),
                      np.sum(np.cos(angles), axis=axis))


@dataclasses.dataclass(frozen=True)
class SimulationResult:
    """A helper class to hold the results of running a simulation."""
    sim: 'Simulator'  # Simulator that generated this result
    infer: Optional[Inference]  # Inference procedure that generated this result
    true_poses: Sequence[Pose]
    estimated_poses: Optional[Sequence[Pose]] = None
    # History snapshots for the Inference, for animation purpose
    snapshots: Optional[Sequence[Inference]] = None


@dataclasses.dataclass(frozen=False)
class Simulator:
    """A simulator for a point robot in R^2.

    The simulator simulates the motion and sensor noises of the robot by 
    sampling from 2D Gaussians.
    The simulation has a list of landmarks, each with a unique identifier.
    The sensor of the robot can sense, for each landmark in the robot's sensing
    range, a noise-corrupted distance and bearing to that landmarks (together
    with the identifier that identifies the landmark).
    """

    # Initial simulator state
    init_state: Pose = Pose(0, 0, 0)

    # 2x2 covariance matrix of the motion noise.
    motion_noise_covariance: np.ndarray = np.diag([1e-3, np.deg2rad(3)**2])

    # 2x2 covariance matrix of the sensor noise.
    sensor_noise_covariance: np.ndarray = np.diag([1, np.deg2rad(5)**2])

    # A set of landmarks
    landmarks: Sequence[Landmark] = grid_of_landmarks()

    # Robot can only sense landmarks within this range
    max_sensing_range: float = np.inf

    # Current state of the robot
    state: Pose = dataclasses.field(init=False, default=None)

    # Random seed control
    rng: np.random.Generator = dataclasses.field(init=False, default=None)
    # Using the same seed will induce the same trajectory
    seed: dataclasses.InitVar[int] = 0

    def __post_init__(self, seed: int):
        assert (len({l.id for l in self.landmarks
                    }) == len(self.landmarks)), "Landmark must have unique IDs"
        self.rng = np.random.default_rng(seed)

    def init(self) -> None:
        self.state = self.init_state

    def simulate_motion(self, command: Command) -> None:
        """Simulate robot motion. This function updates the value of
        self.state.

        Args:
          command: a Command tuple containing fields delta_p(float), the 
          distance of the movement, and delta_theta(float), the rotation of 
          the movement.
        """
        noise = self.rng.multivariate_normal(np.zeros(2),
                                             self.motion_noise_covariance)
        x = (self.state.x + math.cos(self.state.theta) *
             (command.delta_p + noise[0]))
        y = (self.state.y + math.sin(self.state.theta) *
             (command.delta_p + noise[0]))
        theta = self.state.theta + command.delta_theta + noise[1]
        self.state = Pose(x, y, theta)

    def simulate_sensing(self) -> Sequence[Measurement]:
        """Simulate the robot sensing process. This function returns a list of
        measurements, in the same order as the self.landmarks list.
        Specifically, it measures the distance and the angle w.r.t. each
        landmark, and add a Gaussian noise on the measurement.

        Returns:
          measurements: a list of measurements.
        """
        measurements = []
        # Simulate that the measurements' order is random
        for landmark in sorted(self.landmarks, key=lambda _: random.random()):
            r = math.hypot(landmark.x - self.state.x, landmark.y - self.state.y)

            if r >= self.max_sensing_range:
                # Can't sense this landmark, skip
                continue

            b = math.atan2(landmark.y - self.state.y,
                           landmark.x - self.state.x) - self.state.theta
            # Normalize angle to -pi to pi
            b = math.atan2(math.sin(b), math.cos(b))

            noise = self.rng.multivariate_normal(np.zeros(2),
                                                 self.sensor_noise_covariance)

            measurements.append(
                Measurement(landmark.id, r + noise[0], b + noise[1]))

        return measurements

    def run(self,
            commands: Iterable[Command],
            infer: Optional[Inference] = None,
            snapshot_every_n: Optional[int] = 0) -> SimulationResult:
        """Run the simulation with a sequence of commands. Optionally, execute
        an inference algorithm along way with the simulation.

        Args:
            commands: an interable of commands.
            infer: the inference algorithm to run the simulation with.
            snapshot_every_n: snapshot the inference algorithm every n steps.
                if <= 0, don't take any snapshot. 

        Returns:
            SimulationResult
        """
        true_poses, est_poses, snapshots = [], None, None

        self.init()

        i = 0
        true_poses.append(self.state)
        if infer:
            est_poses, snapshots = [], []
            infer.init(self.init_state)
            est_poses.append(infer.estimated_pose())
            if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                snapshots.append(copy.deepcopy(infer))

        for command in commands:
            i += 1
            # Simulate motion
            self.simulate_motion(command)
            true_poses.append(self.state)
            if infer:
                # Simulate measurement
                measurements = self.simulate_sensing()
                infer.update(command, measurements)
                est_poses.append(infer.estimated_pose())
                if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                    snapshots.append(copy.deepcopy(infer))

        return SimulationResult(self, copy.deepcopy(infer), true_poses,
                                est_poses, snapshots)


def drive_in_a_square_commands() -> Iterable[Command]:
    """Commands to simulate the robot's movement in a square."""
    for _ in range(4):
        for i in range(100):
            yield Command(.1, 0)
        yield Command(0, np.pi / 2)


def patrol_commands() -> Iterable[Command]:
    """Commands that walks back and forth horizontally."""
    for i in range(10):
        yield Command(0.5, 0)
    yield Command(0, np.pi)





def run_inference(sim: Simulator,
                  commands: Sequence[Command] = tuple(
                      drive_in_a_square_commands()),
                  mode: str = "localization",
                  num_particles: int = 100,
                  snapshot_every_n=0) -> SimulationResult:
    """Utility to run inference on a simulator with a particular inference mode.

    Args:
        sim: the simulator instance to run the simulation.
        commands: see `Simulator.run`.
        mode: specified the mode of the inference; one of "localization", "SLAM" or "RBSLAM".
            You must implement the corresponding inference algorithm in order 
            to perform inference in any mode.
        num_particles: number of particles of the particle filter.
        snapshot_every_n: see `Simulator.run`.

    Returns:
        A simulation result returned by `Simulator.run`.
    """

    mode = mode.lower()
    try:
        if mode == "localization":
            infer = Localization(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                landmarks=sim.landmarks,
                num_particles=num_particles,
            )
        elif mode == "slam":
            infer = SLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        elif mode == "rbslam":
            infer = RBSLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        else:
            raise ValueError(f"Unrecoginized inference: {mode}")
    except NotImplementedError:
        raise NotImplementedError(
            f"You must completely implement the {infer.__class__} class to "
            f"run simulation under {mode} mode")

    return sim.run(commands, infer=infer, snapshot_every_n=snapshot_every_n)


def plot_samples(x, y, fov=((-6, -6), (20, 20)), show=True):
    """Plots a list of samples of x, y coordinates as input and scatter plot the samples.

    Args:
        x: array of shape (N, ), the x-coordinates of the pose component of the particles.
        y: array of shape (N, ), the y-coordinates of the pose component of the particles.
        fov: a rectangular field of view. The function plots within this 
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(x, y, 'g+')

    if show:
        plt.show()

    return ax


def plot_trajectories(true_poses: Sequence[Pose],
                      estimated_poses: Sequence[Pose],
                      landmarks: Optional[Sequence[Landmark]] = None,
                      ax: Optional['matplotlib.axes.Axes'] = None,
                      fov=((-6, -6), (20, 20)),
                      show=True):
    """Plot true and estimated poses on the same plot by overlaying them.
    It ignores the pose.theta and only plots the location.

    Args:
        true_poses: a sequence of true poses.
        estimated_poses: a sequence of estimated poses.
        landmarks: optinally plot the location of true landmarks.
        ax: an optional matplotlib Axes on which to plot the trajectories.
        fov: a rectangular field of view. The function plots within this 
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    if landmarks:
        ax.scatter([l.x for l in landmarks], [l.y for l in landmarks],
                   marker="o",
                   color="black")

    ax.plot([p.x for p in true_poses], [p.y for p in true_poses],
            color=(0, 0, 1, 0.5))
    ax.plot([p.x for p in estimated_poses], [p.y for p in estimated_poses],
            color=(1, 0, 0, 0.5))

    if show:
        plt.show()
    return ax


def plot_simulation_result(result: SimulationResult,
                           ax: Optional['matplotlib.axes.Axes'] = None,
                           fov=((-6, -6), (20, 20)),
                           title=None,
                           show=True):
    """Plot the inference result (the state at last step of the simulation result).

    Args:
        result: result of a simulation.
        ax: an optional matplotlin Axes to draw the result.
        fov: see `plot_trajectories`.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    if title:
        ax.set_title(title)

    plot_trajectories(result.true_poses,
                      result.estimated_poses,
                      landmarks=result.sim.landmarks,
                      ax=ax,
                      fov=fov,
                      show=False)

    result.infer.plot_state(ax)

    if show:
        plt.show()

    return ax


def render_animation(
    result: SimulationResult,
    fov=((-6, -6), (20, 20)),
    save_file: Optional[str] = None,
) -> 'matplotlib.animation.FuncAnimation':
    """Render simulation as a matplotlib animation.

    Args:
        result: result of a simulation.
        fov: see `plot_trajectories`.
        save_file: an optional string file name to save the animation. 

    Returns:
        an instance of `matplotlib.animation.FuncAnimation`. You can do further 
        processing on the returned animation, display it on Colab, or save it
        as an mp4 file.  
    """
    if result.snapshots is None:
        raise ValueError("SimulationResult does not have any snapshots. "
                         "Please re-run simulation with snapshot_every_n > 0.")

    import matplotlib.animation as animation
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    snapshot_every_n = len(result.true_poses) // len(result.snapshots)
    true_poses = result.true_poses[::snapshot_every_n]
    estimated_poses = result.estimated_poses[::snapshot_every_n]

    def render_frame(i):
        ax.clear()
        ax.set_title(f"Step {i}")
        plot_trajectories(true_poses[:i + 1],
                          estimated_poses[:i + 1],
                          landmarks=result.sim.landmarks,
                          ax=ax,
                          fov=fov,
                          show=False)
        result.snapshots[i].plot_state(ax)
        return ax

    anim = animation.FuncAnimation(fig, render_frame, len(result.snapshots))
    if save_file:
        anim.save(f"{save_file}.mp4", fps=60)
    return anim





## Warmup 1


### Question
Let's make sure we know how to sample from a set 
of values based on their weights.

Please write a function that:
- Takes in a list of samples and associated weights.
- Re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

_Hint:_ You should use the function `numpy.random.choice`.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_1(samples: np.ndarray, weights: np.ndarray,
             nr_samples: int) -> np.ndarray:
    """Draw N samples from the input list based on their weights.

    Args:
      samples: a numpy array of shape (n_samples, ), a list of samples.
      weights: a numpy float array of shape (n_samples, ), indicating 
        the weights for individual samples.
      nr_samples: an integer, indicating the number of samples to draw.

    Returns:
      resampled_samples: the return numpy array, containing nr_samples
        samples drawn from the input list, based on the weights with replacement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.03191091, 0.10365558, 0.07293406, 0.13085621, 0.12995933, 0.0454193 , 0.04606439, 0.1336077 , 0.15964489, 0.14594763], dtype=np.float64), 10), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4], dtype=np.int64), atol=1e-6)
print('Tests passed.')

## Warmup 2


### Question
We are going to be sampling from Gaussian distributions in this miniproject. 
First, let us check that we know how to sample from a multivariate Gaussian.
Please write a function that takes in a mean vector, a covariance matrix and a integer number 
of samples and returns a list of the requested number of samples drawn from a Gaussian with the given mean and covariance.

_Hint:_ You should use the function `numpy.random.multivariate_normal`.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_2(mean: np.ndarray, covariance: np.ndarray,
             num_particles: int) -> np.ndarray:
    """Sample from a multi-variate Gaussian distribution.

    Args:
      mean: a numpy array of shape (N, ).
      covariance: a numpy array of shape (N, N).
      num_particles: an integer.

    Returns:
      sample: a numpy vector with length num_particles.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([0.58345008, 0.05778984, 0.13608912], dtype=np.float64), np.array([[1.61405906, 0.93399482, 0.79165733],  [0.93399482, 1.65599579, 1.43616931],  [0.79165733, 1.43616931, 1.25177385]], dtype=np.float64), 5), np.array([[-0.8347847 , -2.30256418, -1.83805061],  [-0.13115218, -3.33917428, -2.91354857],  [-0.47476854, -1.05885092, -0.83435982],  [ 0.29510331, -0.55660856, -0.28675913],  [-0.06919861, -0.94320128, -0.69794801]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## Localization: Update by Motion Model


### Utilities

Particle Filter for Localization
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

try:
    from functools import cached_property
except ImportError:
    # Import for Colab (Python==3.7)
    from backports.cached_property import cached_property


def multivariate_normal_logpdf(xs: np.ndarray, means: np.ndarray,
                               covs: np.ndarray) -> np.ndarray:
    """Batch version of `scipy.stats.multivariate_normal.logpdf`.

    Compute multivariate normal's log probability density function, with batch 
    inputs and distributions.
    Modified from https://gregorygundersen.com/blog/2020/12/12/group-multivariate-normal-pdf/. 

    This function uses the following broadcasting logic:
    - First, `means` and `covs` are broadcasted together, which determines `dist_batch`, 
        the number of individual multivariate normal distributions. 
    - Then, broadcast the rightmost dimensions of `xs` to `means`. The rest of the dimensions 
    of `xs` becomes the `input_batch` dimension.

    Args:
        xs: array of shape (...input_batch,  ...dist_batch, dist_dim)
        means: array of shape (...dist_batch, dist_dim)
        covs: array of shape (...dist_batch, dist_dim, dist_dim)

    Returns:
        array of shape (...input_batch, ...dist_batch, dist_dim)


    Below we describe some uses cases, with different input shapes and the result shape.

    1.  xs: (100, 2), means: (2,),  covs: (2, 2), result: (100, 2)
        Explanation: 
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=(100,)` and `dist_batch=()`
            - returns logpdf of 100 inputs, evaluated by the same mean and covariance.

    2.  xs: (100, 2), means: (2,),  covs: (100, 2, 2), result: (100, 2)
        Explanation: 
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=()` and `dist_batch=(100,)`
            - returns logpdf of 100 inputs, evaluated by the same mean but with 100 different covariances.
    """
    dist_dim = xs.shape[-1]
    dist_batch_shape = np.broadcast_shapes(means.shape[:-1], covs.shape[:-2])
    means = np.broadcast_to(means, dist_batch_shape + (dist_dim,))
    covs = np.broadcast_to(covs, dist_batch_shape + (dist_dim, dist_dim))
    xs = np.broadcast_to(xs, xs.shape[:-len(means.shape)] + means.shape)

    vals, vecs = np.linalg.eigh(covs)
    logdets = np.sum(np.log(vals), axis=-1)
    valsinvs = 1. / vals
    Us = vecs * np.sqrt(valsinvs)[..., np.newaxis, :]
    devs = xs - means
    devUs = np.einsum("...i,...ij->...j", devs, Us)
    mahas = np.sum(np.square(devUs), axis=-1)
    log2pi = np.log(2 * np.pi)
    out = -0.5 * (dist_dim * log2pi + mahas + logdets)
    return out


@dataclasses.dataclass(frozen=True)
class LocalizationParticles:
    """A batch of particles."""

    x: np.ndarray  # shape (N, )
    y: np.ndarray  # shape (N, )
    theta: np.ndarray  # shape (N, )

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert len(self.x.shape) == 1
        # Always normalize the angles
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of particles."""
        return self.x.shape[0]


@dataclasses.dataclass
class Localization(Inference):
    """Localization Inference by particle filtering."""

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # In localization, we assume that we have access to the set of landmarks.
    landmarks: List[Landmark] = dataclasses.field(default_factory=list)

    num_particles: int = 10

    particles: LocalizationParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    @cached_property
    def landmarks_id_map(self) -> Dict[str, Landmark]:
        """Map from unique landmark id to the Landmark instance."""
        return {l.id: l for l in self.landmarks}

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        # we know the landmarks so no estimation required!
        return self.landmarks

    def init(self, init_state: Pose) -> None:
        self.particles = LocalizationParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.4),
        )

    # We incrementally implement methods below using the `implementation_for` helper
    def motion_model(self, command: Command):
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command, measurements: Sequence[Measurement]):
        raise NotImplementedError()


def plot_particles(samples: LocalizationParticles, fov=((-6, -6), (20, 20))):
    """matplotlib helper function. It takes a list of samples as input and
    scatter plot the samples.

    Args:
        particles: a set of particles for localization. This function only plots 
            the first two components: x and y.
        fov: a rectangular field of view. The function plots within this 
            rectanglular fov.
    """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(samples.x, samples.y, 'g+')
    plt.show()

### Question

Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

**Note**
In this project, we will use a helper decorator `implementation_for` to implement
class methods outside of the class. 
The decorator `@implementation_for(Localization)` simply inserts the declared
function as a method of the Localization class. 
Using this decorator, you copy only the implemented method into 
catsoop's solution box and avoid repeating other parts of the class definition.


**Hint**: Use `np.random.multivariate_normal` to generate the noise.


For reference, our solution is **10** line(s) of code.

In [ ]:
@implementation_for(Localization)
def motion_model(self: Localization, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot, and
    updates to the new samples.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def localization_motion_model_problem_test():
    # A simulator with no noise
    sim = Simulator(motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)))
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = Localization(
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        landmarks=sim.landmarks,
        num_particles=num_particles,
    )
    pf.init(sim.state)
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"

localization_motion_model_problem_test()

print('Tests passed.')

## Localization: Computing the Importance Weights


### Question

Let us now add in a measurement model. Let us assume that
there are four landmarks located at $(x, y)$ locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
relative bearing $[r, b]$ from the robot's location to each of these landmarks. If the
robot's pose is $(x, y, \theta)$ and the landmark position is $(l_x, l_y)$,
then range is just the Euclidean distance $$r = \sqrt{(l_x - x)^2 + (l_y
-y)^2}$$, 
and the bearing is the angle to the landmark in the robot's body frame (hence
relative bearing), $$b = \texttt{atan2}(l_y - y, l_x - x) - \theta.$$  Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Please take a look at our implementation in `simulator.simulate_sensing` and
understand how the values are simulated.

Now the problem is that for a real robot, even if you know this is the model
that generates the measurements, you can't know the noise that nature applied. (That would be omniscience!) 
What we can do is work out how likely each measurement is. More precisely, given a set of measurements, we can use
this model to compute the importance weights of a set of samples of the robot
pose. See [Lecture 13 Page 13](https://sicp-s4.mit.edu/_static/fall22/lectures/lec13/L13-Sampling.pdf#page=13)
for details.

In particular, to compute the importance weight of a particle:
- Compute the _predicted measurements_ from the current state of the robot's pose and the landmark locations.
- Compute the errors $\mathit{err}$ between the predicted and true measurements.
- Compute the likelihood of the $\mathit{err}$ as the unnormalized importance weights. 
The likelihood of the error is the evaluation of the probability density function (PDF) of the two-dimensional Gaussian distribution with covariance `sensor_noise_covariance` at $\mathit{err}$.
- Finally, normalize the weights so that they sum to one. 

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.

There are a few technicalities during this process:
- We highly recommend you normalize **all the rotational values** (in radians) to the interval $[-\pi, \pi]$. 
Notably, the rotational component of the $\mathit{err}$, which we denote as $\mathit{err}_\theta$, is a rotational 
value in radians. Therefore, theoretically, the likelihood of $\mathit{err}_\theta$ should have been
the evaluation of the PDF of a [wrapped Gaussian distribution](https://en.wikipedia.org/wiki/Wrapped_normal_distribution). 
In other words, to evaluate the likelihood of 
$\mathit{err}_\theta$, we should have taken the sum of the Gaussian PDFs at the points 
$\mathit{err}_\theta + 2k\pi$ for all $k \in \mathbb{Z}$. 
Nonetheless, since our sensor noise is relatively small, we expect only the single point at $k=0$
$\mathit{err}_\theta \in [-\pi, \pi]$ to have a relatively large value, and all other
points to have close-to-zero values. Therefore, we may use the Gaussian distribution as a 
good approximation to the true wrapped Gaussian distribution.
- The likelihood for the particles can become really small. For numerical stability, 
you will want to compute the _log-likelihood_ instead of the likelihood. 
To that end, you may:
    - Use the utility `multivariate_normal_logpdf` that we provided you to evaluate the log PDF. 
    - Take the sum (instead of multiplications) of the log-likelihoods to compute the unnormalized log-weight of each particle.
    - Normalize the log-weights. You should use [`scipy.special.logsumexp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html) to evaluate 
the log-sum of the weights. In short, The `logsumexp` function calculates the following expression in a more 
numerically stable way:
$$
\texttt{logsumexp}(x_1, ... x_n) = \log \left( \sum_{i=1}^{n} \exp(x_i) \right).
$$ See <a href="https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/" target="_blank">this blog</a> for a more detailed description of this function.
    - Use `numpy.exp` to undo the logarithm from the log-weights and return the weights.


For reference, our solution is **29** line(s) of code.

In [ ]:
@implementation_for(Localization)
def compute_weights(self: Localization,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem2_test(compute_weights, infer,
                           measurements: Sequence[Measurement],
                           result_weights: np.ndarray):
    weights = compute_weights(infer, measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

sampling_problem2_test(compute_weights, Localization(motion_noise_covariance=np.array([[0.01      , 0.        ],  [0.        , 0.00761544]], dtype=np.float64), sensor_noise_covariance=np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64), landmarks=(Landmark(id='landmark-0', x=-4, y=-4), Landmark(id='landmark-1', x=-4, y=0), Landmark(id='landmark-2', x=-4, y=4), Landmark(id='landmark-3', x=-4, y=8), Landmark(id='landmark-4', x=-4, y=12), Landmark(id='landmark-5', x=-4, y=16), Landmark(id='landmark-6', x=0, y=-4), Landmark(id='landmark-7', x=0, y=0), Landmark(id='landmark-8', x=0, y=4), Landmark(id='landmark-9', x=0, y=8), Landmark(id='landmark-10', x=0, y=12), Landmark(id='landmark-11', x=0, y=16), Landmark(id='landmark-12', x=4, y=-4), Landmark(id='landmark-13', x=4, y=0), Landmark(id='landmark-14', x=4, y=4), Landmark(id='landmark-15', x=4, y=8), Landmark(id='landmark-16', x=4, y=12), Landmark(id='landmark-17', x=4, y=16), Landmark(id='landmark-18', x=8, y=-4), Landmark(id='landmark-19', x=8, y=0), Landmark(id='landmark-20', x=8, y=4), Landmark(id='landmark-21', x=8, y=8), Landmark(id='landmark-22', x=8, y=12), Landmark(id='landmark-23', x=8, y=16), Landmark(id='landmark-24', x=12, y=-4), Landmark(id='landmark-25', x=12, y=0), Landmark(id='landmark-26', x=12, y=4), Landmark(id='landmark-27', x=12, y=8), Landmark(id='landmark-28', x=12, y=12), Landmark(id='landmark-29', x=12, y=16), Landmark(id='landmark-30', x=16, y=-4), Landmark(id='landmark-31', x=16, y=0), Landmark(id='landmark-32', x=16, y=4), Landmark(id='landmark-33', x=16, y=8), Landmark(id='landmark-34', x=16, y=12), Landmark(id='landmark-35', x=16, y=16)), num_particles=20, particles=LocalizationParticles(x=np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=np.float64), y=np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=np.float64), theta=np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=np.float64))), [Measurement(landmark_id='landmark-35', r=22.753147219062917, b=0.7738698392857971), Measurement(landmark_id='landmark-25', r=12.640422650443282, b=0.009154262150241616), Measurement(landmark_id='landmark-15', r=8.408602536838048, b=1.1387038391295914), Measurement(landmark_id='landmark-3', r=10.248271955129297, b=2.117092341243524), Measurement(landmark_id='landmark-32', r=15.78868726666365, b=0.1345498076509456), Measurement(landmark_id='landmark-12', r=5.033579786955029, b=-0.7817917913663452), Measurement(landmark_id='landmark-7', r=-2.3250307746388343, b=-0.01909317455769924), Measurement(landmark_id='landmark-20', r=7.698360962746094, b=0.399745227278584), Measurement(landmark_id='landmark-5', r=15.948163519613333, b=1.7881725941556874), Measurement(landmark_id='landmark-2', r=6.068484785866513, b=2.4471709441565186), Measurement(landmark_id='landmark-26', r=12.520575977729484, b=0.44099698774324514), Measurement(landmark_id='landmark-24', r=11.983915967186904, b=-0.291075514011446), Measurement(landmark_id='landmark-8', r=4.903470181651809, b=1.5790004474613593), Measurement(landmark_id='landmark-30', r=15.748923253116834, b=-0.32541437620132907), Measurement(landmark_id='landmark-11', r=15.542274174332661, b=1.5900119763018357), Measurement(landmark_id='landmark-4', r=11.639492457134782, b=1.8742928687102223), Measurement(landmark_id='landmark-34', r=19.84077499008552, b=0.6906987897814902), Measurement(landmark_id='landmark-9', r=8.21465912250634, b=1.6018084460172888), Measurement(landmark_id='landmark-27', r=13.76837649243762, b=0.5766916802305041), Measurement(landmark_id='landmark-14', r=6.44082971955371, b=0.9157246165771075), Measurement(landmark_id='landmark-23', r=16.629478287894198, b=1.2392634902611783), Measurement(landmark_id='landmark-21', r=12.659583922767066, b=0.8535804455254041), Measurement(landmark_id='landmark-13', r=4.264455630329303, b=-0.027394933553940907), Measurement(landmark_id='landmark-1', r=5.458020683536959, b=3.312657462648057), Measurement(landmark_id='landmark-6', r=5.801634869866125, b=-1.4560318732949584), Measurement(landmark_id='landmark-33', r=18.245924230657273, b=0.3582019162682134), Measurement(landmark_id='landmark-18', r=8.939817776879076, b=-0.40635936363131575), Measurement(landmark_id='landmark-0', r=4.368492785742826, b=-2.3217135857051487), Measurement(landmark_id='landmark-31', r=16.42986369482223, b=0.06074118633851044), Measurement(landmark_id='landmark-22', r=13.23808713509877, b=0.9250492804923485), Measurement(landmark_id='landmark-19', r=7.563564752856779, b=-0.10208447443373762), Measurement(landmark_id='landmark-10', r=13.739367877130134, b=1.5275199517585014), Measurement(landmark_id='landmark-16', r=12.97808027013372, b=1.2264810610293408), Measurement(landmark_id='landmark-28', r=18.554035627279266, b=0.9006213960947502), Measurement(landmark_id='landmark-29', r=20.633352622824916, b=0.7350027054140555), Measurement(landmark_id='landmark-17', r=16.54445147673053, b=1.3854805390653757)], np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05], dtype=np.float64))
print('Tests passed.')

## Localization: Particle Filter Update


### Question

What we want you to do is to write a method for step 4: `update` that takes in a
set of samples, the most recent command, and the most recent measurements, and then:
  * Use the motion_model method to update the sample based on the command
  * Use compute_weights to compute the sample weights from the
measurement,
  * Resample a new set of samples,
  * Blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag([1e-3, 1e-3, np.deg2rad(1)**2])`. (You'll need to
recall from lecture why we do this blurring --- if you have machine learning
background, it's a form of regularization, but you don't need to know about
regularization to understand why this is important.)


For reference, our solution is **16** line(s) of code.

In [ ]:
@implementation_for(Localization)
def update(self: Localization, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def sampling_problem3_test(_, infer, command: Command,
                           measurements: Sequence[Measurement], pose: Pose):
    infer.update(command, measurements)
    est_pose = infer.estimated_pose()
    print(est_pose)
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.3), "Estimated pose should be quite close to our estimation"

sampling_problem3_test(update, Localization(motion_noise_covariance=np.array([[0.0001, 0.    ],  [0.    , 0.0001]], dtype=np.float64), sensor_noise_covariance=np.array([[1.00000000e-06, 0.00000000e+00],  [0.00000000e+00, 9.27917724e-08]], dtype=np.float64), landmarks=(Landmark(id='landmark-0', x=-4, y=-4), Landmark(id='landmark-1', x=-4, y=0), Landmark(id='landmark-2', x=-4, y=4), Landmark(id='landmark-3', x=-4, y=8), Landmark(id='landmark-4', x=-4, y=12), Landmark(id='landmark-5', x=-4, y=16), Landmark(id='landmark-6', x=0, y=-4), Landmark(id='landmark-7', x=0, y=0), Landmark(id='landmark-8', x=0, y=4), Landmark(id='landmark-9', x=0, y=8), Landmark(id='landmark-10', x=0, y=12), Landmark(id='landmark-11', x=0, y=16), Landmark(id='landmark-12', x=4, y=-4), Landmark(id='landmark-13', x=4, y=0), Landmark(id='landmark-14', x=4, y=4), Landmark(id='landmark-15', x=4, y=8), Landmark(id='landmark-16', x=4, y=12), Landmark(id='landmark-17', x=4, y=16), Landmark(id='landmark-18', x=8, y=-4), Landmark(id='landmark-19', x=8, y=0), Landmark(id='landmark-20', x=8, y=4), Landmark(id='landmark-21', x=8, y=8), Landmark(id='landmark-22', x=8, y=12), Landmark(id='landmark-23', x=8, y=16), Landmark(id='landmark-24', x=12, y=-4), Landmark(id='landmark-25', x=12, y=0), Landmark(id='landmark-26', x=12, y=4), Landmark(id='landmark-27', x=12, y=8), Landmark(id='landmark-28', x=12, y=12), Landmark(id='landmark-29', x=12, y=16), Landmark(id='landmark-30', x=16, y=-4), Landmark(id='landmark-31', x=16, y=0), Landmark(id='landmark-32', x=16, y=4), Landmark(id='landmark-33', x=16, y=8), Landmark(id='landmark-34', x=16, y=12), Landmark(id='landmark-35', x=16, y=16)), num_particles=100, particles=LocalizationParticles(x=np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=np.float64), y=np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=np.float64), theta=np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=np.float64))), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id='landmark-4', r=12.649236370894611, b=1.8925066397489418), Measurement(landmark_id='landmark-16', r=12.649751063323961, b=1.249077726801277), Measurement(landmark_id='landmark-20', r=8.943736240625999, b=0.46375775715344025), Measurement(landmark_id='landmark-35', r=22.628720998014654, b=0.7856866607567657), Measurement(landmark_id='landmark-34', r=19.999296264764194, b=0.6431156393698316), Measurement(landmark_id='landmark-10', r=11.999376725537463, b=1.5708089154080955), Measurement(landmark_id='landmark-12', r=5.654529218717742, b=-0.7854648111495861), Measurement(landmark_id='landmark-7', r=-0.0012459109472530653, b=-0.00022306139218380394), Measurement(landmark_id='landmark-33', r=17.887999561015462, b=0.4635512584632948), Measurement(landmark_id='landmark-17', r=16.492834133007015, b=1.3261352314007253), Measurement(landmark_id='landmark-8', r=3.999871465337056, b=1.5712125753715283), Measurement(landmark_id='landmark-31', r=15.999334805326514, b=0.00010707609058078074), Measurement(landmark_id='landmark-1', r=4.000903470181652, b=3.141621291373365), Measurement(landmark_id='landmark-29', r=19.999256500750647, b=0.9270144443957444), Measurement(landmark_id='landmark-15', r=8.943814184173492, b=1.1072157930644515), Measurement(landmark_id='landmark-5', r=16.491412884287104, b=1.815711271397861), Measurement(landmark_id='landmark-18', r=8.944112684989244, b=-0.46348285801429606), Measurement(landmark_id='landmark-25', r=12.000214659122506, b=0.00010825271769035358), Measurement(landmark_id='landmark-23', r=17.8878899913889, b=1.1071092352234257), Measurement(landmark_id='landmark-21', r=11.314492474454822, b=0.7858530885395344), Measurement(landmark_id='landmark-11', r=15.998740934467897, b=1.5712574943489113), Measurement(landmark_id='landmark-32', r=16.493768377894426, b=0.24521666418979546), Measurement(landmark_id='landmark-28', r=16.970827204107472, b=0.7853025370396719), Measurement(landmark_id='landmark-30', r=16.49388052315418, b=-0.24438153429639173), Measurement(landmark_id='landmark-27', r=14.424006736725824, b=0.5884032070631326), Measurement(landmark_id='landmark-3', r=8.944629290409818, b=2.0340758608916567), Measurement(landmark_id='landmark-22', r=14.422200647722839, b=0.9829936969482067), Measurement(landmark_id='landmark-0', r=5.655565888028631, b=-2.3560741291298712), Measurement(landmark_id='landmark-14', r=5.6572841131872025, b=0.785610190136083), Measurement(landmark_id='landmark-24', r=12.647926522706761, b=-0.32195212052680294), Measurement(landmark_id='landmark-2', r=5.656417814245238, b=2.3558381481535373), Measurement(landmark_id='landmark-6', r=4.00173936787713, b=-1.5709473898414392), Measurement(landmark_id='landmark-9', r=8.00032896962946, b=1.5707175610932667), Measurement(landmark_id='landmark-19', r=8.001583472878803, b=0.00040220495707190147), Measurement(landmark_id='landmark-26', r=12.649743993296344, b=0.3210793269023251), Measurement(landmark_id='landmark-13', r=4.00005202897426, b=0.00020826272337815132)], Pose(x=0.9858299248258179, y=-0.00020686081759629056, theta=0.7682645354557489))
print('Tests passed.')

## SLAM: Update by Motion Model


### Utilities

SLAM Particle Filter
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


@dataclasses.dataclass(frozen=True)
class SLAMParticles:
    """A batch of samples for SLAM."""
    x: np.ndarray  # shape: (num_particles, )
    y: np.ndarray  # shape: (num_particles, )
    theta: np.ndarray  # shape: (num_particles, )

    # Mapping from landmark_id to the index of that landmark in `landmarks_loc`
    landmarks_id_to_idx: Dict[str, int]

    landmarks_loc: np.ndarray  # shape (num_particles, num_landmarks, 2)

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert self.x.shape[0] == self.landmarks_loc.shape[0]
        assert self.landmarks_loc.shape[1] == len(self.landmarks_id_to_idx)
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of samples."""
        return self.x.shape[0]


@dataclasses.dataclass
class SLAM(Inference):

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # A 2x2 array for the covariance when initializing new landmarks
    new_landmarks_init_covariance: np.ndarray = np.eye(2) * 10

    num_particles: int = 10

    particles: SLAMParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        return [
            Landmark(idx_to_id[i], *loc) for i, loc in enumerate(landmarks_loc)
        ]

    def init(self, init_state: Pose) -> None:
        self.particles = SLAMParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
            {},
            np.empty((self.num_particles, 0, 2), dtype=np.float64),
        )

    def get_measured_landmarks_idx(
            self, measurements: Sequence[Measurement]) -> np.ndarray:
        """Given a sequence of measurements, find the indices of each of the l
        andmark in `self.particles`.

        Args:
            measurements: a sequence of measures. This function assumes that all
            landmarks in this input sequence of measurements are already tracked 
            (i.e., using `self.add_new_landmarks`).
        """
        return np.array(
            [
                self.particles.landmarks_id_to_idx[m.landmark_id]
                for m in measurements
            ],
            dtype=int,
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        import matplotlib.pyplot as plt
        cmap = plt.get_cmap('viridis')

        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.1),
        )

        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        landmark_idx_to_color_code = np.array([
            float(hash(idx_to_id[idx]) % 256) / 256
            for idx in range(self.particles.landmarks_loc.shape[1])
        ])

        ax.scatter(self.particles.landmarks_loc[..., 0].flatten(),
                   self.particles.landmarks_loc[..., 1].flatten(),
                   marker="x",
                   c=np.broadcast_to(
                       landmark_idx_to_color_code,
                       self.particles.landmarks_loc.shape[:-1]).flatten(),
                   cmap=cmap)

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> None:
        raise NotImplementedError()

    def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

### Question

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$ 
and updates the particles by drawing drawn from the posterior $X_{t+1}$ using the motion model. 

This function behaves the same as `Localization.motion_model` for the pose 
components of the particles, and simply keeps the landmarks components 
(`SLAMParticles.landmarks_id_to_idx` and `SLAMParticles.landmarks_loc`) 
of the particles the same.

**Hint**

Please take a look at the fields of the `SLAMParticles` class.
Note that `landmarks_id_to_idx` stores all the landmarks the robot has encountered so far.
It is a mapping from a landmark identifier (a string) to an index (an integer). 
You can use the integer index to index into the `landmarks_loc` array. 
In particular, `landmarks_loc` is an array of shape `(num_particles, len(landmarks_id_to_idx), 2)`.
Therefore, to access the coordinate of the landmark with the identifier `"Stata"` contained
in the $i$-th particle:
```python
# assert isinstance(particles, SLAMParticles)
loc = particles.landmarks_loc[i, particles.landmarks_id_to_idx["Stata"]]
```


For reference, our solution is **12** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def motion_model(self: SLAM, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot and updates the particles.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

## SLAM: Tracking New Landmarks


### Question

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we have to match newly observed landmarks to ones we have seen before, 
and add landmarks that we are seeing for the first time to our map.  
In some situations, the matching is difficult, but for our purposes, 
we’ll assume that the landmarks are uniquely identifiable, so we know exactly 
which ones we are observing on each step.

One way to handle missing data due to sensing range is to explicitly model the 
sensing range in our probabilistic model of the sensors. 
Howewer, this approach would induce a difficult inference problem.  
Instead, we will take the approach of treating the missingness as uninformative.
In particular, we will consider sensor data within-range as observed variable, 
and sensor data out-of-range as unobserved variables.
When we see a new landmark for the first time, we will assume that our belief 
about the location of that landmark has a mean equal to that first observed location 
and a standard variance (new_landmarks_init_covariance).
Note, though, that in practice it might be more robust to wait until we have gotten 
a few observations and use their mean as the mean of the distribution for initializing our particles.

Please implement a function that takes a set of measurements and updates the 
particles to include potentially new landmarks that are not yet tracked by the 
particles. 

Below we give a description for this procedure:

- Filter the input measurements to contain only measurements to the new (untracked) 
landmarks. 
- For each new landmark measurement and each particle:
    - Find the predicted location using the robot's pose of that particle and the landmark's measurement.
    - Add a Gaussian noise of covariance `self.new_landmarks_init_covariance` to this predicted location.
    - This noisy location becomes the landmark's location in the current particle.
- Update `self.particles.landmarks_id_to_idx` to maintain the invariance that it 
holds a mapping from each tracked unique landmark indentifier to the index of 
that landmark in `self.particles.landmarks_loc`. 


For reference, our solution is **31** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def add_new_landmarks(self: SLAM, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added 
        before).
    """
    new_measurements = [
        m for m in measurements
        if m.landmark_id not in self.particles.landmarks_id_to_idx
    ]

    # Insert new landmarks into the mapping `landmarks_id_to_idx`
    new_landmarks_id_to_idx = dict(self.particles.landmarks_id_to_idx)
    for m in new_measurements:
        new_landmarks_id_to_idx[m.landmark_id] = len(
            new_landmarks_id_to_idx)

    # Find estimated landmark location
    # based on the new measurements and the current pose in the particles
    est_landmarks_loc = ...  # shape: (num_particles, |new_measurements|, 2)

    # Sample new landmark locations around est_landmarks_loc with
    # zero-mean Gaussian noise of `new_landmarks_init_covariance``
    sampled_landmark_locs = ...  # shape: (num_particles, |new_measurements|, 2)

    # Update particles
    new_particles = dataclasses.replace(
        self.particles,
        landmarks_id_to_idx=new_landmarks_id_to_idx,
        landmarks_loc=np.concatenate(
            [self.particles.landmarks_loc, sampled_landmark_locs], axis=1),
    )

    raise NotImplementedError("Implement me!")

    self.particles = new_particles

## SLAM: Computing the Importance Weights


### Question

Please implement the `SLAM.compute_weights` function that takes a 
sequence of measurements and returns the importance weights of the particles.

This function is similar to `Localization.compute_weights`, except that it takes 
into account the landmarks location tracked in the particles.


For reference, our solution is **30** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def compute_weights(self: SLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the particles, based on the measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

## SLAM: Particle Filter Update


### Question

Please implement the update step of SLAM.

The procedure is similar to `Localization.update`, except that it does an additional `add_new_landmarks` 
step after forward updating with the `motion_model` and before `compute_weights`. 


For reference, our solution is **20** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def update(self: SLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the particles, based on the command and measurements.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError("Implement me!")

## Einsum Warmup (Optional)


### Question

This question intends to get you familiar with using `numpy.einsum`.
**This question is optional and you do not have to use `einsum` in your implementation.** 
However, using `einsum` can greatly simplify your implementation.

In particular, the Rao-blackwellized particle filter performs quite a few 
matrix multiplications. 
The function `numpy.einsum` allows one to express batched multiplication of 
multiple matrices succinctly.

If you are not familar with `np.einsum`, here are some guides online:
- <a href="https://stackoverflow.com/questions/26089893/understanding-numpys-einsum" target="_blank">StackOverflow Post</a>
- <a href="https://rockt.github.io/2018/04/30/einsum" target="_blank">Blog: einsum is all you need</a>

For this warmup question, please fill in the indexing expression string for 
`einsum` to implement a 'sandwich matrix product'. 
A sandwich matrix product of two matrices $U$ and $S$ is
$M = U \cdot S \cdot U^\top$.


For reference, our solution is **2** line(s) of code.

In [ ]:
def sandwich_matrix_product(U: np.ndarray, S: np.ndarray) -> np.ndarray:
    """Perform a batched 'sandwitch matrix product': $$M = U \cdot S \cdot U^\top$$.
    Uses `numpy.einsum`.

    Args:
        U, S: input matrices to perform the product, both of shape (batch, N, N)

    Returns:
        the matrices M, of shape (batch, N, N)
    """
    return np.einsum("FILL IN THE STRING HERE", U, S, U)

### Tests

In [ ]:

assert np.allclose(sandwich_matrix_product(np.array([[[-0.91509102,  0.02896068,  0.09373636],   [-0.31979851, -0.86280633, -0.5421848 ],   [-0.28403213, -0.12971603,  0.18185345]],   [[ 0.44478304, -0.36473625, -0.34209248],   [-0.96061671, -0.91825028, -0.48435661],   [ 0.48049   ,  0.25662766,  0.53957804]],   [[ 0.53783887,  0.71313494,  0.44063853],   [ 0.95802184,  0.79765044,  0.17343433],   [ 0.17631534, -0.93146592,  0.99705316]],   [[-0.73684801,  0.48069439,  0.64203039],   [-0.25389094, -0.60629589, -0.80248023],   [ 0.49721201, -0.09469294,  0.42743552]],   [[ 0.8308153 , -0.70683253,  0.838342  ],   [-0.17674708, -0.38946598,  0.88612452],   [ 0.98130339, -0.60221556,  0.31367669]]], dtype=np.float64), np.array([[[-0.78700937,  0.30182801,  0.65462646],   [ 0.36899709, -0.16533372, -0.23386728],   [-0.21375517,  0.17942364,  0.76313454]],   [[ 0.85813231, -0.89294076, -0.63675521],   [-0.77555137, -0.61333072, -0.30678438],   [ 0.01306337,  0.25892245,  0.46428444]],   [[ 0.78022308,  0.97817687,  0.32571296],   [ 0.69072904,  0.55607769, -0.38493592],   [ 0.75138454, -0.91447372, -0.99926531]],   [[-0.45253474, -0.07580494,  0.27672579],   [-0.79645947,  0.34602027,  0.60363173],   [-0.62937416, -0.16974949,  0.03996998]],   [[-0.09638596,  0.59965986,  0.9210448 ],   [ 0.59790633, -0.84401436,  0.60987114],   [-0.86680734, -0.52805923, -0.69380621]]], dtype=np.float64)), np.array([[[-0.70821093,  0.29028311, -0.264794  ],   [-0.0860032 ,  0.25683661, -0.0839603 ],   [-0.12767688, -0.00731765, -0.0378081 ]],   [[ 0.5021111 , -0.23356351,  0.07177661],   [-0.75393822, -1.39959106,  0.63624027],   [ 0.30684435,  0.26353458, -0.08116496]],   [[ 0.80153175,  1.51206662, -0.80293068],   [ 1.56577159,  2.31436395, -1.05422099],   [-1.02144813, -0.87038954,  0.63539018]],   [[ 0.46041849, -0.04331244, -0.18835761],   [-1.13941324,  0.12874572,  0.35331832],   [ 0.30230967,  0.11473338, -0.15291035]],   [[-1.68979947, -0.25599802, -1.6767293 ],   [-1.49516034, -0.63011489, -1.13628625],   [-0.99438737,  0.04839135, -1.17364301]]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## RBSLAM: Update by Motion Model


### Utilities

Rao-blackwellized Particle Filter for SLAM
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


@dataclasses.dataclass(frozen=True)
class RBSLAMParticles:
    """A batch of particles for Rao-blackwellized SLAM."""
    x: np.ndarray  # shape: (num_particles, )
    y: np.ndarray  # shape: (num_particles, )
    theta: np.ndarray  # shape: (num_particles, )

    # Mapping from landmark_id to the index of that landmark in `landmarks_loc`
    landmarks_id_to_idx: Dict[str, int]

    landmarks_loc: np.ndarray  # shape: (num_particles, num_landmarks, 2)
    landmarks_cov: np.ndarray  # shape: (num_particles, num_landmarks, 2, 2)

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert (self.landmarks_loc.shape + (2,) == self.landmarks_cov.shape)
        assert self.landmarks_loc.shape[1] == len(self.landmarks_id_to_idx)
        assert self.landmarks_loc.shape[0] == self.x.shape[0]
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of particles."""
        return self.x.shape[0]


@dataclasses.dataclass
class RBSLAM(Inference):
    """Rao-blackwellized particle filter for SLAM."""

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # A 2x2 array for the covariance when initializing new landmarks
    # By default, we simply use a large enough symmetric covariance
    new_landmarks_init_covariance: np.ndarray = np.eye(2) * 10

    num_particles: int = 10

    particles: RBSLAMParticles = None

    # To account for the approximation error of EKF when a landmark gets really
    # close, we will ignore any measurement that is within min_sensing_range!
    min_sensing_range: float = 1.

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        return [
            Landmark(idx_to_id[i], *loc) for i, loc in enumerate(landmarks_loc)
        ]

    def init(self, init_state: Pose) -> None:
        self.particles = RBSLAMParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
            {},
            np.empty((self.num_particles, 0, 2), dtype=np.float64),
            np.empty((self.num_particles, 0, 2, 2), dtype=np.float64),
        )

    def get_measured_landmarks_idx(
            self, measurements: Sequence[Measurement]) -> np.ndarray:
        """Given a sequence of measurements, find the indices of each of the 
        landmark in `self.particles`.

        Args:
            measurements: a sequence of measures. This function assumes that all
            landmarks in this input sequence of measurements are already tracked 
            (i.e., using `self.add_new_landmarks`).
        """
        return np.array(
            [
                self.particles.landmarks_id_to_idx[m.landmark_id]
                for m in measurements
            ],
            dtype=int,
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        import matplotlib.pyplot as plt

        cmap = plt.get_cmap('viridis')

        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.1),
        )

        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        landmark_idx_to_color_code = np.array([
            float(hash(idx_to_id[idx]) % 256) / 256
            for idx in range(self.particles.landmarks_loc.shape[1])
        ])

        ax.scatter(self.particles.landmarks_loc[..., 0].flatten(),
                   self.particles.landmarks_loc[..., 1].flatten(),
                   marker="x",
                   c=np.broadcast_to(
                       landmark_idx_to_color_code,
                       self.particles.landmarks_loc.shape[:-1]).flatten(),
                   cmap=cmap)

        avg_landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        avg_landmarks_cov = np.mean(self.particles.landmarks_cov, axis=0)
        for i, (loc, cov) in enumerate(zip(avg_landmarks_loc,
                                           avg_landmarks_cov)):
            edgecolor = cmap(landmark_idx_to_color_code[i])
            confidence_ellipse(loc, cov, ax, n_std=5, edgecolor=edgecolor)

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> None:
        raise NotImplementedError()

    def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def update_EKF(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def compute_weights(self, measurements: Sequence[Measurement],
                        sensor_noise_covariance: np.ndarray) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()


def confidence_ellipse(loc, cov, ax, n_std=3.0, facecolor="none", **kwargs):
    """Draws a confidence ellipse for a 2D Gaussian."""

    from matplotlib.patches import Ellipse
    import matplotlib.transforms as transforms

    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
                      width=ell_radius_x * 2,
                      height=ell_radius_y * 2,
                      facecolor=facecolor,
                      **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std

    # calculating the stdandard deviation of y
    scale_y = np.sqrt(cov[1, 1]) * n_std

    transf = transforms.Affine2D()\
        .rotate_deg(45)\
        .scale(scale_x,scale_y)\
        .translate(*loc)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

### Question

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$ 
and updates the particles by drawing drawn from the posterior $X_{t+1}$ using the 
motion model. 

This function behaves the same as `Localization.motion_model` for the pose 
components of the particles, and simply keeps the landmarks components 
(`landmarks_id_to_idx`, `landmarks_loc` and `landmarks_cov`) of the 
particles the same.


For reference, our solution is **9** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def motion_model(self: RBSLAM, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot and updates the particles.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

## RBSLAM: Tracking New Landmarks


### Question

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we must track new landmarks that appears in the measurements in our 
particles.

Please implement `add_new_landmarks` that takes a sequence of measurements and updates 
the particles to include potentially new landmarks that are not yet tracked by the 
particles. 

When you encounter a new landmark, instead of generating a set of particles to represent a distribution (as in particle filter for SLAM section), 
you should store that landmark into each particle.
For each particle, the new landmark location's distribution 
- is centered the observed location of the landmark (relative to the particle's robot pose).
- has a fixed initial covariance of `self.new_landmarks_init_covariance`.


For reference, our solution is **31** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added 
        before).
    """
    raise NotImplementedError("Implement me!")

## RBSLAM: Extended Kalman Filter Update


### Question

Please implement `update_EKF` that takes a sequence of measurements and performs
an extended Kalman filter update on the landmarks of the particles.

Suppose our robot's pose is $\texttt{Pose}(x, y, \theta)$.
The state that we would like to track by our filter is the landmark location $[\hat{l_x}, \hat{l_y}]$.
Recall that our observation model (without noise) is 
$$
\mathit{Obs}\left(\begin{bmatrix} l_x \\ l_y \end{bmatrix}\right) = \begin{bmatrix} 
\sqrt{(l_x - x)^2 + (l_y -y)^2} \\   
\texttt{atan2}(l_y - y, l_x - x) - \theta
\end{bmatrix}.
$$
This model is non-linear, and the standard Kalman filter does not apply.
This is where an extended Kalman filter could be useful.

In our case, an EKF update is the same as a Kalman filter update.
The idea of EKF is to approximate $\mathit{Obs}$ by its Jacobian around the current belief of location of the landmark, $[\hat{l_x}, \hat{l_y}]$
, so that we get a linear approximation:
$$
\mathit{Obs}\left(\begin{bmatrix} l_x \\ l_y \end{bmatrix}\right) \approx 
\nabla\mathit{Obs}\left(\begin{bmatrix} \hat{l_x} \\ \hat{l_y} \end{bmatrix}\right) \cdot \begin{bmatrix} l_x \\ l_y \end{bmatrix}
= \begin{bmatrix} 
\frac{\hat{l_x} - x}{r} & \frac{\hat{l_y} - y}{r} \\   
-\frac{\hat{l_y} - y}{r^2} & \frac{\hat{l_x} - x}{r^2}
\end{bmatrix} \cdot \begin{bmatrix} l_x \\ l_y \end{bmatrix} \triangleq H \cdot \begin{bmatrix} l_x \\ l_y \end{bmatrix} 
$$, where $r = \sqrt{(\hat{l_x} - x)^2 + (\hat{l_y} - y)^2}$.
Therefore, the implementation of an EKF for our application is almost the same as a KF, 
where we use the matrix $H$ above for the observation model. 


For reference, our solution is **36** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def update_EKF(self, measurements: Sequence[Measurement]) -> None:
    """Performs an extended Kalman filter update on the landmarks.

    Args:
        measurements: the sequence of landmark measurements.
    """
    raise NotImplementedError("Implement me!")

## RBSLAM: Computing the Importance Weights


### Question

Please implement a function that takes a set of particles and measurements and
returns the importance weights of the particles. 


For reference, our solution is **44** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def compute_weights(self: RBSLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the particles, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

## RBSLAM: Particle Filter Update


### Question

Please implement the update step of RB SLAM.

The procedure is similar to `SLAM.update`, except it does an additional `update_EKF` step after `add_new_landmarks` and before `compute_weights`. 

Further, since by using EKF we approximated the non-linear observation model by a linear model, the approximation can be quite bad
when the robot is very close to a landmark. 
To fix, we will simply ignore any landmark measurement that is within `min_sensing_range`!
Don't panic! If everything is implemented correctly, you will see that even when we ignore information of nearby landmarks, 
we still get a much better result than using a plain particle filter for SLAM. 


For reference, our solution is **27** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def update(self: RBSLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    # Filter out any measurement that gets too close,
    # since they mess up with our EKF updates
    measurements = [m for m in measurements if m.r > self.min_sensing_range]

    raise NotImplementedError("Implement me!")